#                                 <font color=Black>TRIP ADVISER Review Analysis </font>

## Data Source

 As part of this analysis, we analyze the Hotel reviews from **Tripadviser**. we could find the [Trip Adviser dataset here](https://github.com/kavgan/OpinRank). This analysis is focused to **Text Graph** technique.

In [71]:
# Required Packages

from collections import Counter, defaultdict
from scipy.sparse import lil_matrix, spmatrix, csr_matrix, save_npz, load_npz
import matplotlib.pyplot as plt
import numpy as np
import string
import os
import re
import pandas as pd
import sys
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import re

from gensim.models import Word2Vec
import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk

import datetime
np.random.seed(2018)

In [74]:
#Selecting cities and data path
DataPath="E:\\Techi\\Courses\\Masters in Data Science\\Courses\\Text Visualization\\Data sets\\Trip Advisor\\hotels\\data\\"
Cities=['new-delhi','beijing', 'chicago', 'dubai', 'las-vegas', 'london', 'montreal',  'new-york-city', 'san-francisco', 'shanghai']
Stopwords=["nan","quot","amp","a", "about", "above", "after", "again", "against", "ain", "all", "am", "an", "and", "any", "are", "aren", "aren't", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "couldn", "couldn't", "d", "did", "didn", "didn't", "do", "does", "doesn", "doesn't", "doing", "don", "don't", "down", "during", "each", "few", "for", "from", "further", "had", "hadn", "hadn't", "has", "hasn", "hasn't", "have", "haven", "haven't", "having", "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "ll", "m", "ma", "me", "mightn", "mightn't", "more", "most", "mustn", "mustn't", "my", "myself", "needn", "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "re", "s", "same", "shan", "shan't", "she", "she's", "should", "should've", "shouldn", "shouldn't", "so", "some", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", "through", "to", "too", "under", "until", "up", "ve", "very", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "when", "where", "which", "while", "who", "whom", "why", "will", "with", "won", "won't", "wouldn", "wouldn't", "y", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves", "could", "he'd", "he'll", "he's", "here's", "how's", "i'd", "i'll", "i'm", "i've", "let's", "ought", "she'd", "she'll", "that's", "there's", "they'd", "they'll", "they're", "they've", "we'd", "we'll", "we're", "we've", "what's", "when's", "where's", "who's", "why's", "would", "able", "abst", "accordance", "according", "accordingly", "across", "act", "actually", "added", "adj", "affected", "affecting", "affects", "afterwards", "ah", "almost", "alone", "along", "already", "also", "although", "always", "among", "amongst", "announce", "another", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "apparently", "approximately", "arent", "arise", "around", "aside", "ask", "asking", "auth", "available", "away", "awfully", "b", "back", "became", "become", "becomes", "becoming", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "believe", "beside", "besides", "beyond", "biol", "brief", "briefly", "c", "ca", "came", "cannot", "can't", "cause", "causes", "certain", "certainly", "co", "com", "come", "comes", "contain", "containing", "contains", "couldnt", "date", "different", "done", "downwards", "due", "e", "ed", "edu", "effect", "eg", "eight", "eighty", "either", "else", "elsewhere", "end", "ending", "enough", "especially", "et", "etc", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "except", "f", "far", "ff", "fifth", "first", "five", "fix", "followed", "following", "follows", "former", "formerly", "forth", "found", "four", "furthermore", "g", "gave", "get", "gets", "getting", "give", "given", "gives", "giving", "go", "goes", "gone", "got", "gotten", "h", "happens", "hardly", "hed", "hence", "hereafter", "hereby", "herein", "heres", "hereupon", "hes", "hi", "hid", "hither", "home", "howbeit", "however", "hundred", "id", "ie", "im", "immediate", "immediately", "importance", "important", "inc", "indeed", "index", "information", "instead", "invention", "inward", "itd", "it'll", "j", "k", "keep", "keeps", "kept", "kg", "km", "know", "known", "knows", "l", "largely", "last", "lately", "later", "latter", "latterly", "least", "less", "lest", "let", "lets", "like", "liked", "likely", "line", "little", "'ll", "look", "looking", "looks", "ltd", "made", "mainly", "make", "makes", "many", "may", "maybe", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "million", "miss", "ml", "moreover", "mostly", "mr", "mrs", "much", "mug", "must", "n", "na", "name", "namely", "nay", "nd", "near", "nearly", "necessarily", "necessary", "need", "needs", "neither", "never", "nevertheless", "new", "next", "nine", "ninety", "nobody", "non", "none", "nonetheless", "noone", "normally", "nos", "noted", "nothing", "nowhere", "obtain", "obtained", "obviously", "often", "oh", "ok", "okay", "old", "omitted", "one", "ones", "onto", "ord", "others", "otherwise", "outside", "overall", "owing", "p", "page", "pages", "part", "particular", "particularly", "past", "per", "perhaps", "placed", "please", "plus", "poorly", "possible", "possibly", "potentially", "pp", "predominantly", "present", "previously", "primarily", "probably", "promptly", "proud", "provides", "put", "q", "que", "quickly", "quite", "qv", "r", "ran", "rather", "rd", "readily", "really", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "respectively", "resulted", "resulting", "results", "right", "run", "said", "saw", "say", "saying", "says", "sec", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sent", "seven", "several", "shall", "shed", "shes", "show", "showed", "shown", "showns", "shows", "significant", "significantly", "similar", "similarly", "since", "six", "slightly", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "specifically", "specified", "specify", "specifying", "still", "stop", "strongly", "sub", "substantially", "successfully", "sufficiently", "suggest", "sup", "sure", "take", "taken", "taking", "tell", "tends", "th", "thank", "thanks", "thanx", "thats", "that've", "thence", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "thereto", "thereupon", "there've", "theyd", "theyre", "think", "thou", "though", "thoughh", "thousand", "throug", "throughout", "thru", "thus", "til", "tip", "together", "took", "toward", "towards", "tried", "tries", "truly", "try", "trying", "ts", "twice", "two", "u", "un", "unfortunately", "unless", "unlike", "unlikely", "unto", "upon", "ups", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "v", "value", "various", "'ve", "via", "viz", "vol", "vols", "vs", "w", "want", "wants", "wasnt", "way", "wed", "welcome", "went", "werent", "whatever", "what'll", "whats", "whence", "whenever", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "whim", "whither", "whod", "whoever", "whole", "who'll", "whomever", "whos", "whose", "widely", "willing", "wish", "within", "without", "wont", "words", "world", "wouldnt", "www", "x", "yes", "yet", "youd", "youre", "z", "zero", "a's", "ain't", "allow", "allows", "apart", "appear", "appreciate", "appropriate", "associated", "best", "better", "c'mon", "c's", "cant", "changes", "clearly", "concerning", "consequently", "consider", "considering", "corresponding", "course", "currently", "definitely", "described", "despite", "entirely", "exactly", "example", "going", "greetings", "hello", "help", "hopefully", "ignored", "inasmuch", "indicate", "indicated", "indicates", "inner", "insofar", "it'd", "keep", "keeps", "novel", "presumably", "reasonably", "second", "secondly", "sensible", "serious", "seriously", "sure", "t's", "third", "thorough", "thoroughly", "three", "well", "wonder", "a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "around", "as", "at", "back", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "co", "op", "research-articl", "pagecount", "cit", "ibid", "les", "le", "au", "que", "est", "pas", "vol", "el", "los", "pp", "u201d", "well-b", "http", "volumtype", "par", "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a1", "a2", "a3", "a4", "ab", "ac", "ad", "ae", "af", "ag", "aj", "al", "an", "ao", "ap", "ar", "av", "aw", "ax", "ay", "az", "b1", "b2", "b3", "ba", "bc", "bd", "be", "bi", "bj", "bk", "bl", "bn", "bp", "br", "bs", "bt", "bu", "bx", "c1", "c2", "c3", "cc", "cd", "ce", "cf", "cg", "ch", "ci", "cj", "cl", "cm", "cn", "cp", "cq", "cr", "cs", "ct", "cu", "cv", "cx", "cy", "cz", "d2", "da", "dc", "dd", "de", "df", "di", "dj", "dk", "dl", "do", "dp", "dr", "ds", "dt", "du", "dx", "dy", "e2", "e3", "ea", "ec", "ed", "ee", "ef", "ei", "ej", "el", "em", "en", "eo", "ep", "eq", "er", "es", "et", "eu", "ev", "ex", "ey", "f2", "fa", "fc", "ff", "fi", "fj", "fl", "fn", "fo", "fr", "fs", "ft", "fu", "fy", "ga", "ge", "gi", "gj", "gl", "go", "gr", "gs", "gy", "h2", "h3", "hh", "hi", "hj", "ho", "hr", "hs", "hu", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ic", "ie", "ig", "ih", "ii", "ij", "il", "in", "io", "ip", "iq", "ir", "iv", "ix", "iy", "iz", "jj", "jr", "js", "jt", "ju", "ke", "kg", "kj", "km", "ko", "l2", "la", "lb", "lc", "lf", "lj", "ln", "lo", "lr", "ls", "lt", "m2", "ml", "mn", "mo", "ms", "mt", "mu", "n2", "nc", "nd", "ne", "ng", "ni", "nj", "nl", "nn", "nr", "ns", "nt", "ny", "oa", "ob", "oc", "od", "of", "og", "oi", "oj", "ol", "om", "on", "oo", "oq", "or", "os", "ot", "ou", "ow", "ox", "oz", "p1", "p2", "p3", "pc", "pd", "pe", "pf", "ph", "pi", "pj", "pk", "pl", "pm", "pn", "po", "pq", "pr", "ps", "pt", "pu", "py", "qj", "qu", "r2", "ra", "rc", "rd", "rf", "rh", "ri", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "rv", "ry", "s2", "sa", "sc", "sd", "se", "sf", "si", "sj", "sl", "sm", "sn", "sp", "sq", "sr", "ss", "st", "sy", "sz", "t1", "t2", "t3", "tb", "tc", "td", "te", "tf", "th", "ti", "tj", "tl", "tm", "tn", "tp", "tq", "tr", "ts", "tt", "tv", "tx", "ue", "ui", "uj", "uk", "um", "un", "uo", "ur", "ut", "va", "wa", "vd", "wi", "vj", "vo", "wo", "vq", "vt", "vu", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y2", "yj", "yl", "yr", "ys", "yt", "zi", "zz"]
Custom_Stopwords=["nan","quot","hotel","room"]

## Data Retrivel and Processing

Below code collect reviews from all files and append it.

In [6]:
Dataset=pd.read_csv(DataPath+"text.txt", names=['Date','Title','Content'], delimiter="\t", index_col=3, encoding = "ISO-8859-1")
Dataset["City"]="Test"
Dataset.reset_index()
for city in Cities :
    print("Started : "+city)
    for i in os.listdir(DataPath+city+"\\"):
        try :
            Dataset2=pd.read_csv(DataPath+city+"\\"+i, names=['Date','Title','Content'], delimiter="\t", index_col=3, encoding = "ISO-8859-1")
            Dataset2.reset_index()
            Dataset2["City"]=city
            Dataset2["Hotel"]=i
            Dataset=pd.concat([Dataset,Dataset2],ignore_index=True,axis=0, sort=True)
        except :
            print(city+"\\"+i+" file is corrupted")
    print("Completed : "+city)


Started : new-delhi
Completed : new-delhi
Started : beijing
Completed : beijing
Started : chicago
chicago\usa_illinois_chicago_the_whitehall_hotel file is corrupted
Completed : chicago
Started : dubai
dubai\are_dubai_towers_rotana_dubai file is corrupted
Completed : dubai
Started : las-vegas
Completed : las-vegas
Started : london
london\uk_england_london_best_western_phoenix_hotel file is corrupted
london\uk_england_london_hilton_london_green_park file is corrupted
london\uk_england_london_merlyn_court_hotel file is corrupted
london\uk_england_london_st_david_s_hotels file is corrupted
Completed : london
Started : montreal
Completed : montreal
Started : new-york-city
new-york-city\usa_new york city_park_central file is corrupted
Completed : new-york-city
Started : san-francisco
Completed : san-francisco
Started : shanghai
Completed : shanghai


## Extracting date features

In [7]:
Dataset["Date"]=pd.to_datetime(Dataset["Date"])
Dataset.index=Dataset.Date
Dataset["Year"]=Dataset.index.year
Dataset["Month"]=Dataset.index.month

## Extracting hotel ratings

In [11]:
city_review=pd.read_csv(DataPath+"city_text.txt", header=0 , delimiter="\t",encoding = "ISO-8859-1")
for city in Cities :
   
    try :
        city_review2=pd.read_csv(DataPath+city+".csv",header=0 , delimiter=",",  encoding = "ISO-8859-1",index_col=False)
        city_review=pd.concat([city_review,city_review2],ignore_index=True,axis=0, sort=True)
    except :
        print(city+"\\"+i+" file is corrupted")
city_review['overall_ratingsource']=city_review['overall_ratingsource'].replace(-1,np.NaN)
#city_review[['doc_id','country','overall_ratingsource']]

## Data Preprocess Function

In [158]:
# Taken from Gensim preparators
RE_PUNCT = re.compile(r'([%s])+' % re.escape(string.punctuation), re.UNICODE)
RE_NUMERIC = re.compile(r"[0-9]+", re.UNICODE)
RE_NONALPHA = re.compile(r"\W", re.UNICODE)
RE_WHITESPACE = re.compile(r"(\s)+", re.UNICODE)
MIN_LENGTH = 3

def normalise(s):
    if s is None or s is np.nan:
        return ""
    s = s.lower()
    s = RE_PUNCT.sub(" ", s)
    s = RE_WHITESPACE.sub(" ", s)
    s = RE_NUMERIC.sub("", s)
    s = ' '.join([w for w in s.split() if len(w)>=MIN_LENGTH])
    s = RE_NONALPHA.sub(" ", s)
    return s
    
#1 for skip-gram; otherwise CBOW.
def create_model(Sentences2, Model_Name, sg1=0):
    
    print('Creating the model is starting : '+str(datetime.datetime.now()))
    model_review = Word2Vec(sentences=list(pd.Series(Sentences2).dropna()) , size=100, window=5, min_count=1, workers=4, sg=sg1)
    print('Creating the model is completed : '+str(datetime.datetime.now()))
    if sg1==0 :
        model_review.save("model\\"+Model_Name+"_"+"word2vec.model")
    else :
        model_review.save("model\\"+Model_Name+"_"+"word2vec_Skipgram.model")
    print('Model is Saved : '+str(datetime.datetime.now()))
    return model_review

def load_model_file(modelfilename, list_of_words, sg1=0):
    if sg1==0:
        model= Word2Vec.load("model\\"+modelfilename+"_"+"word2vec.model")
    else :    
        model= Word2Vec.load("model\\"+modelfilename+"_"+"word2vec_Skipgram.model")
    word1=[]
    year=[]
    s_word=[]
    s_word_rate=[]
    for x in list_of_words:
        try :
            for sw in model.wv.similar_by_word(x) :
                word1.append(x)
                year.append(np.nan)
                s_word.append(sw[0])
                s_word_rate.append(sw[1])
        except :
            pass
    similiar_item=pd.DataFrame({"word": word1, "s_word":s_word, "s_word_rate":s_word_rate}) 
    if sg1==0:
        similiar_item["Type"]=modelfilename
    else :
        similiar_item["Type"]=modelfilename+"_sg"
    return similiar_item

sno = nltk.stem.SnowballStemmer('english')
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(sno.stem(token))
    return result

def create_sentence(rule):
    print('Reading the sentence Started : '+str(datetime.datetime.now()))
#    print(len(Dataset["Content"][rule]))
    Sentences=[]
    for review in Dataset["Content"][rule]:
        if pd.isnull(review)== False and review!='':
            tokens = preprocess(review)
            tokens=[x for x in tokens if x not in Custom_Stopwords]
            Sentences.append(tokens)
            
            #print(tokens)
    print('Reading the sentence Completed : '+str(datetime.datetime.now()))
    print('Total Sentences '+str(len(Sentences)))
    return Sentences

def corpora_dict_model(sentence1):
    dictionary = corpora.Dictionary(sentence1)
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in sentence1]
    print('Creating the model is starting : '+str(datetime.datetime.now()))
    # Running and Trainign LDA model on the document term matrix.
    ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, #passes=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    print('Creating the model is completed : '+str(datetime.datetime.now()))
    return (doc_term_matrix,ldamodel)

def samplesenteneces(Sentences1, limit=1000):
    if (len(Sentences1) >= limit) :
        print('Before restricting (size) : '+str(len(Sentences1)))
        Sentences1=list(pd.Series(Sentences1).dropna()[np.linspace(0,len(Sentences1),limit,dtype='int')])
        print('After restricting (size) : '+str(len(Sentences1)))
        return list(pd.Series(Sentences1).dropna())
    else :
        print('Sentence size already with in a limit (size) : '+str(len(Sentences1)))
        return list(pd.Series(Sentences1).dropna())

In [75]:
highlowrated_sentence=create_sentence( Dataset['Hotel'].isin(list(city_review[(city_review['overall_ratingsource']<=2) | (city_review['overall_ratingsource']>=4.7)]['doc_id'])) )
dictionary = corpora.Dictionary(highlowrated_sentence)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in highlowrated_sentence]
print('Creating the model is starting : '+str(datetime.datetime.now()))
# Running and Training LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=4, id2word = dictionary, #passes=50, 
                                       random_state=100,
                                       update_every=1,
                                       chunksize=100,
                                       passes=10,
                                       alpha='auto',
                                       per_word_topics=True)

print('Creating the model is completed : '+str(datetime.datetime.now()))

Reading the sentence Started : 2019-05-18 18:50:16.864354
Reading the sentence Completed : 2019-05-18 18:50:27.270950
Total Sentences 3939
Creating the model is starting : 2019-05-18 18:50:28.462018
Creating the model is completed : 2019-05-18 18:51:49.180635


In [275]:
#Most common words
Dataset["NormalisedContent"] = Dataset["Content"].apply(normalise)
full_vocab = defaultdict(int)
i=0
for review in Dataset["NormalisedContent"] :
    i=i+1
    if review!="":
        for token in review.split():
            full_vocab[token] += 1

list_of_impo_words=[]
full_vocab_c=Counter(full_vocab)

In [265]:
list_of_impo_words=[]
for (w,s) in full_vocab_c.most_common(50):
    if w not in Stopwords :
        list_of_impo_words.append(w)

In [266]:
similiars=load_model_file("delhi", list_of_impo_words, 1)

london_similiar=load_model_file("london", list_of_impo_words, 1)
similiars=pd.concat([similiars,london_similiar],ignore_index=True,axis=0, sort=True)

newyork_similiar=load_model_file("newyork", list_of_impo_words, 1)
similiars=pd.concat([similiars,newyork_similiar],ignore_index=True,axis=0, sort=True)

### Pre-analyzed topic 

-   Topic 0 : Sight seeing    
-   Topic 1 : Hostel    
-   Topic 2 : Food    
-   Topic 3 : Staff Service

In [276]:
All_node_words=list(set(list(similiars["s_word"])+list_of_impo_words))

313

In [268]:
impo_wtopic_id=[]
impo_wtopic_name=[]
for wtopic in All_node_words:
    try :
        topic_id=ldamodel[dictionary.doc2bow([wtopic])][1][0][1][0]
        if topic_id==0 :
            topic_name="Sight seeing"
        elif topic_id==1 :
            topic_name="Hostel"
        elif topic_id==2 :
            topic_name="Food"
        elif topic_id==3 :
            topic_name="Staff Service"
        
        impo_wtopic_id.append(topic_id)
        impo_wtopic_name.append(topic_name)
    except :
        impo_wtopic_id.append(1)
        impo_wtopic_name.append("Hostel")

In [269]:
node=pd.DataFrame({ "id":All_node_words,"Label":All_node_words, "topic_name":impo_wtopic_name})
node.to_csv("node_topic.csv", index=False)

In [270]:
Topic_df=pd.DataFrame({ "word":All_node_words,"topic_id":impo_wtopic_id,  "topic_name":impo_wtopic_name})
similiar_topic=similiars.merge(Topic_df, how="inner")

In [273]:
city="delhi_sg"
Edge=pd.DataFrame({"Source":similiar_topic[similiar_topic["Type"]==city]["word"],"Target":similiar_topic[similiar_topic["Type"]==city]["s_word"],"Type":"Undirected", "Weight" :similiar_topic[similiar_topic["Type"]==city]["s_word_rate"],"Average Degree":"1"})
Edge.to_csv("Edge_"+city+".csv")